In [7]:
import torch
import torchvision
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import cv2
import os
from tqdm import tqdm

REBUILD_DATA = False # set to true to one once, then back to false unless you want to change something in your training data.

class DogsVSCats():
    IMG_SIZE = 50
    CATS = "PetImages/Cat"
    DOGS = "PetImages/Dog"
    TESTING = "PetImages/Testing"
    LABELS = {CATS: 0, DOGS: 1}
    training_data = []

    catcount = 0
    dogcount = 0

    def make_training_data(self):
        for label in self.LABELS:
            for f in tqdm(os.listdir(label)):
                if "jpg" in f:
                    try:
                        path = os.path.join(label, f)
                        img = cv2.imread(path)
                        img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                        self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])  
                        #print(np.eye(2)[self.LABELS[label]])

                        if label == self.CATS:
                            self.catcount += 1
                        elif label == self.DOGS:
                            self.dogcount += 1

                    except Exception as e:
                        pass
                        #print(label, f, str(e))

        np.random.shuffle(self.training_data)
        np.save("training_data_mggy.npy", self.training_data)
        print('Cats:',dogsvcats.catcount)
        print('Dogs:',dogsvcats.dogcount)

if REBUILD_DATA:
    dogsvcats = DogsVSCats()
    dogsvcats.make_training_data()

In [8]:
train=np.load("training_data_mggy.npy", allow_pickle=True)
X = torch.Tensor([i[0] for i in train]).view(-1, 3, 50, 50)
X=X/255.0
y = torch.Tensor([i[1] for i in train])

split=int(len(X)*0.1)

train_X = X[:-split]
train_y = y[:-split]

test_X = X[-split:]
test_y = y[-split:]

print(len(train_X), len(train_y))


22452 22452


In [16]:
from statistics import mean

net=models.resnet18(pretrained=True)
dim_in=net.fc.in_features
net.fc=nn.Linear(dim_in, 2)
net=net.to(device)
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_f = nn.MSELoss()
batch = 80
EPOCHS = 15

acc = []
for epoch in range(EPOCHS):
    right=0
    tots=0
    for i in tqdm(range(0, len(train_X), batch)):
        
        batch_X=train_X[i:i+batch].view(-1,3,50,50).to(device)
        batch_y=train_y[i:i+batch].to(device)
        
        net.zero_grad()       
        outputs=net(batch_X)
        if i%50==0:
            for k in range(len(batch_X)):
                if torch.argmax(batch_y[k])==torch.argmax(outputs[k]):
                    right+=1
                tots+=1
            acc.append(right/tots)
            right=0
            tots=0
        loss= loss_f(outputs, batch_y)
        loss.backward()
        optimizer.step()
    print(round(mean(acc), 3), round(float(loss), 4))
    #print()




100%|████████████████████████████████████████████████████████████████████████████████| 281/281 [00:26<00:00, 10.25it/s]


0.604 0.2454


100%|████████████████████████████████████████████████████████████████████████████████| 281/281 [00:27<00:00, 10.14it/s]


0.642 0.2059


100%|████████████████████████████████████████████████████████████████████████████████| 281/281 [00:28<00:00,  9.87it/s]


...

0.812 0.0795


100%|████████████████████████████████████████████████████████████████████████████████| 281/281 [00:29<00:00,  8.87it/s]


0.821 0.0597


100%|████████████████████████████████████████████████████████████████████████████████| 281/281 [00:29<00:00,  9.04it/s]


0.829 0.0605


In [17]:
correct=0 
total=0

#net3=models.resnet18(pretrained=True)
#dim_in=net3.fc.in_features
#net3.fc=nn.Linear(dim_in, 2)
#net3=net3.to(device)


with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        tgt=torch.argmax(test_y[i]).to(device)
        net_out=net(test_X[i].view(-1,3,50,50).to(device))[0]
        prediction=torch.argmax(net_out)
        if prediction==tgt:
            correct+=1
        total+=1
        
print("Accuracy:", round(correct/total, 3))

100%|█████████████████████████████████████████████████████████████████████████████| 2494/2494 [00:16<00:00, 155.77it/s]


Accuracy: 0.547
